# Configuration & Library

In [8]:
import ssl
from pyngrok import ngrok, conf, installer
import io
from PIL import Image
from flask import Flask, request
import nest_asyncio
from pyngrok import ngrok
import torch
from ultralytics import YOLO

ssl._create_default_https_context = ssl._create_unverified_context
pyngrok_config = conf.get_default()

# API Setup


In [1]:

app = Flask(__name__)
model = YOLO('best.pt')


@app.route("/objectdetection/", methods=["POST"])
def predict():
    if not request.method == "POST":
        return

    if request.files.get("image"):
        image_file = request.files["image"]
        image_bytes = image_file.read()
        img = Image.open(io.BytesIO(image_bytes))
        # img = img.reshape(640,640)
        results = model(img)
        class_indices = results[0].boxes.cls
        indices = class_indices.to(dtype=torch.int).tolist()
        names = ['car', 'house', 'road', 'swimming pool', 'tree', 'yard']
        mapped_names = [names[idx] for idx in indices]
        results_json = {
            "boxes": results[0].boxes.xyxy.tolist(),
            "classes": mapped_names,
            "confidence":results[0].boxes.conf.tolist()
        }
        return {"result": results_json}


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
app.run(host="0.0.0.0", port=8000)

ModuleNotFoundError: No module named 'PIL'

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
